# Earnings Analyzer: A Walkthrough

This notebook provides a simple, step-by-step guide to using the `earnings_analyzer` package. We will walk through the core features, from analyzing a new earnings call to retrieving data from the database.

## Troubleshooting API Key Issues

If you encounter errors while running the analysis, here are common solutions:

In [ ]:
# If you see "GEMINI_API_KEY not found" or "FMP_API_KEY not found":
from earnings_analyzer.config import set_gemini_api_key, set_fmp_api_key

# Set your keys programmatically (good for notebooks)
set_gemini_api_key("your_actual_gemini_api_key_here")
set_fmp_api_key("your_actual_fmp_api_key_here")

# Verify they're set correctly
from earnings_analyzer.config import get_gemini_api_key, get_fmp_api_key
print("Gemini key set:", "Yes" if get_gemini_api_key() else "No")
print("FMP key set:", "Yes" if get_fmp_api_key() else "No")

## 1. Imports and Setup

In [ ]:
import pandas as pd
from earnings_analyzer.analyzer import EarningsAnalyzer
from earnings_analyzer.config import set_gemini_api_key, set_fmp_api_key

## 2. Set API Keys & Initialize Analyzer

In [2]:
# IMPORTANT: Replace with your actual API keys
set_gemini_api_key("YOUR_GEMINI_API_KEY")
set_fmp_api_key("YOUR_FMP_API_KEY")

analyzer = EarningsAnalyzer()

## 3. Analyze a New Earnings Call

In [3]:
nvda_df = analyzer.analyze_to_dataframe('NVDA')

pd.DataFrame(nvda_df)

INFO: Attempting to find latest transcript for NVDA...
INFO: Found existing analysis for NVDA Q4 2025. Returning cached data.


,Ticker,Company Name,Sector,Industry,Sentiment Model,Overall Sentiment Score,Sentiment Confidence,Key Themes,Qualitative Assessment,Price at Call,1 Week Performance,1 Month Performance,3 Month Performance,Call Date,Quarter
0,NVDA,NVIDIA Corporation,Technology,None,gemini-2.5-flash,10.0,0.95,"Explosive Data Center Growth & Blackwell Ramp,...",The executive sentiment during the earnings ca...,131.28,-0.10649,-0.16461,0.032145,2025-02-26,Q4


### Understanding the Caching Feature
The first time you run the analysis for a specific company and quarter, the tool performs a full analysis, including API calls to fetch data and perform sentiment analysis. It then saves this complete record to a local database.

If you run the exact same analysis again, the tool will find the saved record and load it directly from your disk. This is faster and reduces your API calls. You will see a message indicating that the cached data is being used.

## 4. Retrieve Records from the Database

You can easily retrieve the records you have saved. The `get_existing_calls()` method allows you to pull all records for a specific ticker, while `get_all_calls()` retrieves every record in the database.

In [4]:
# Get all records for a specific ticker
nvda_records = analyzer.get_existing_calls('NVDA')

pd.DataFrame(nvda_records)

,ticker,call_date,quarter,year,filing_url,overall_sentiment_score,confidence_level,model_name,qualitative_assessment,key_themes
0,NVDA,2025-02-26,Q4,2025,https://www.fool.com/earnings/call-transcripts...,10.0,0.95,gemini-2.5-flash,The executive sentiment during the earnings ca...,[Explosive Data Center Growth & Blackwell Ramp...


In [5]:

# Get all records from the database
all_calls = analyzer.get_all_calls()

pd.DataFrame(all_calls)

,ticker,call_date,quarter,year,filing_url,overall_sentiment_score,confidence_level,model_name,qualitative_assessment,key_themes
0,NVDA,2025-02-26,Q4,2025,https://www.fool.com/earnings/call-transcripts...,10.0,0.95,gemini-2.5-flash,The executive sentiment during the earnings ca...,[Explosive Data Center Growth & Blackwell Ramp...
1,AAPL,2023-10-01,Q4,2023,https://www.fool.com/earnings/call-transcripts...,10.0,0.95,gemini-2.5-flash,The executive sentiment during the earnings ca...,[Exceptional Software Platform Performance & A...


## 5. Understanding the AI-Powered Metrics

The `earnings_analyzer` uses the Gemini API to provide three key metrics that offer a quick assessment of an earnings call. Here’s a transparent look at what they are and how they are generated.

### Overall Sentiment Score, Confidence Level, Key Themes, and Qualitative Assessment
These metrics are now generated in a single, efficient API call. The tool sends the entire transcript to the Gemini API and requests a comprehensive analysis in a single JSON object. This ensures that the qualitative assessment is a direct analysis of the transcript, providing a more accurate and insightful summary.
**Prompt Used:**
```
As a financial analyst, please analyze the following earnings call transcript.
Based on the language, tone, and key topics discussed by the executives, provide a sentiment analysis.
Return your analysis as a JSON object with the following four fields:
1.  `overall_sentiment_score`: A numerical score from 1 (very negative) to 10 (very positive).
2.  `confidence_level`: Your confidence in this sentiment score, from 0.0 to 1.0.
3.  `key_themes`: A JSON list of the top 3-5 most important themes or topics discussed.
4.  `qualitative_assessment`: A 2-3 sentence qualitative assessment of the executive sentiment during the earnings call, focusing on the overall tone and the implications of the key themes.

Transcript:
---
{transcript_text} # Use the full transcript
---
```